In [ ]:
# 1.loader 
    1.1 

In [5]:
import cv2
import os
import torch
import json
import ipdb
import numpy as np
from torch.utils.data import Dataset
import matplotlib.pyplot as plt

annotation_path = "../chongq/chongqing1_round1_train1_20191223/annotations.json"
img_path = './chongqing1_round1_train1_20191223/images/'


class Jiu2(Dataset):
    def __init__(self):
        
        with open(annotation_path, 'r') as load_f:
            self.load_dict = json.load(load_f)
            
        self.tmp_img_dic = {}
        for dic in self.load_dict['images']:
            idx = dic['id']
            self.tmp_img_dic[idx] = dic
            
        self.id2cls = {}
        for dic in self.load_dict['categories']:
            self.id2cls[dic['id']] = dic['name']
            
    def __len__(self):
        return len(self.load_dict['annotations'][:100])
        
    def __getitem__(self, index):
        # img, mask, C, bbox, h, w
        anno_dic = self.load_dict['annotations'][:100][index]
        img_id = anno_dic['image_id']
        H, W, C = self.tmp_img_dic[img_id]['height'], self.tmp_img_dic[img_id]['width'], anno_dic['category_id']
        mask_img = self.mask_from_bbox(W, H, C, bbox=anno_dic['bbox'])
        img_name = self.tmp_img_dic[img_id]['file_name']
        img = cv2.imread(img_path+img_name).astype(np.float32)/255
        return torch.from_numpy(img).permute([2,0,1]), torch.from_numpy(mask_img), C, torch.tensor(anno_dic['bbox']), H, W
    
    def mask_from_bbox(self, W, H, C, bbox):
        # 图片的x, y轴与np的 x,y轴不一致；需要交换坐标系x,y才能互相转换；
        mask = np.zeros((1, H, W))
        x, y, w, h = [round(i) for i in bbox]
        mask[0, y:y+h, x:x+w] = C
        return mask
    
    
if __name__ == '__main__':
    data = Jiu2()
    

In [6]:
from model import Unet
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler

train_loader = DataLoader(
                    data,
                    shuffle=RandomSampler(data),
                    batch_size=10,
                    num_workers=8,
                    pin_memory=True)

In [7]:
model = Unet()
model.train()
i = 0
for imgs, masks, cls, bbox, H, W in train_loader:
    if i == 0:
        logits = model(imgs)
    i+=1

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /Users/bruce/.cache/torch/checkpoints/resnet34-333f7ec4.pth
 29%|██▉       | 24.3M/83.3M [43:49<1:46:21, 9.69kB/s] 


UnpicklingError: unpickling stack underflow

In [ ]:
def train(train_loader, model):
    running_loss = 0.0
    data_size = train_data.__len__()

    model.train()
    # for inputs, masks, labels in progress_bar(train_loader, parent=mb):
    for inputs, masks, labels in train_loader:
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            if args.is_pseudo:
                logit, logit_pixel, logit_image = model(inputs)
                loss1 = lovasz_hinge(logit.squeeze(1), masks.squeeze(1))
                loss2 = nn.BCELoss()(logit_image, labels)
                loss3 = lovasz_hinge2(logit_pixel.squeeze(1), masks.squeeze(1))
                loss = loss1 + loss2 + loss3
            else:
                logit = model(inputs)
                loss = lovasz_hinge(logit.squeeze(1), masks.squeeze(1))

            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        # mb.child.comment = 'loss: {}'.format(loss.item())
    epoch_loss = running_loss / data_size
    return epoch_loss

In [1]:
from data import *
def iou_thresh(w, h):
    '''
    iou_thresh(120, 400)
    '''
    min_axis = min(w, h)
    if min_axis < 40:
        val = 0.2
    elif (min_axis >= 40) & (min_axis<120):
        val = min_axis/200
    elif (min_axis >= 120) & (min_axis<420):
        val = min_axis/1500 + 0.52
    elif min_axis>=420:
        val = 0.8
    else:
        raise Exception('min_axis:{} not allowed'.format(min_axis))
    return val


def calcIOU(bbox1, bbox2):
    '''
    res  = calcIOU((1, 2, 2, 2), (2, 1, 2, 2))
    '''
    x1,y1,w1,h1 = bbox1
    x2,y2,w2,h2 = bbox2
    if((abs(x1 - x2) < ((w1 + w2)/ 2.0)) and (abs(y1-y2) < ((h1 + h2)/2.0))):
        left = max((x1 - (w1 / 2.0)), (x2 - (w2 / 2.0)))
        upper = max((y1 - (h1 / 2.0)), (y2 - (h2 / 2.0)))

        right = min((x1 + (w1 / 2.0)), (x2 + (w2 / 2.0)))
        bottom = min((y1 + (h1 / 2.0)), (y2 + (h2 / 2.0)))

        inter_w = abs(left - right)
        inter_h = abs(upper - bottom)
        inter_square = inter_w * inter_h
        union_square = (w1 * h1)+(w2 * h2)-inter_square

        calcIOU = inter_square/union_square * 1.0
        return round(calcIOU, 3) 
    else:
        return 0
    return calcIOU

    


def cal_ap(c_pred, c_label, pred_bbox, real_bbox, img_w, img_h):
    '''
    c_pred: [bs];
    c_label: int;
    pred_bbox: list of [x,y,w,h];
    real_bbox: list of [x,y,w,h];
    img_w: [w]
    img_h: [h]
    '''
    
    round1_idx = np.where(c_pred == c_label)[0]
#     ipdb.set_trace()
    pred_list = []
    for idx in round1_idx:
        iou_th = iou_thresh(img_w[idx],img_h[idx])
        is_true = (calcIOU(pred_bbox[idx], real_bbox[idx]) > iou_th) * 1.0
        pred_list.append(is_true)
#     ipdb.set_trace()
    round2_idx = np.where(np.array(pred_list) == 1.0)[0]
    return len(round2_idx)/len(c_pred)


def mAP(weight, c_pred, c_label, pred_bbox, real_bbox, img_w, img_h):
    '''
    weight: {C: weight_value};
    c_pred: [bs];
    c_label: [bs];
    pred_bbox: list of [x,y,w,h];
    real_bbox: list of [x,y,w,h];
    img_w: [w]
    img_h: [h]
    '''
    mAp = 0
    c_label = np.array(c_label)
    for C in range(1,10):
        C_idx = np.where(c_label==C)[0]
#         ipdb.set_trace()
        
        C_ap = cal_ap(np.array(c_pred)[C_idx], np.array(c_label)[C_idx], np.array(pred_bbox)[C_idx], 
                                np.array(real_bbox)[C_idx], np.array(img_w)[C_idx], np.array(img_h)[C_idx])
        mAp+=C_ap*weight[C]
    return mAp
    
    
if __name__ == '__main__':
    
    # iou
    res  = calcIOU((1, 2, 2, 2), (2, 1, 2, 2))
    print(res)
    
    
    # ap & mAP
    c_label_list = np.random.randint(1,11,100)
    c_label = 2
    c_pred = np.random.randint(0,4,100)
    data = Jiu()
    bb1_list = []
    for i in range(100):
        bb1_list.append(data[i][4])
    bb2_list = bb1_list
    img_w = np.random.randint(20,450,100)
    img_h = np.random.randint(20,450,100)
    ap = cal_ap(c_pred, c_label, bb1_list, bb1_list, img_w, img_h)
    print(ap)
    # weight
    weight_dic = {1:0.15,
    2:0.09,
    3:0.09,
    4:0.05,
    5:0.13,
    6:0.05,
    7:0.12,
    8:0.13,
    9:0.07,
    10:0.12}
    map_ = mAP(weight_dic, c_pred, c_label_list, bb1_list, bb1_list, img_w, img_h)
    print(map_)


0.143
0.27
0.0769090909090909
